In [1]:
import os
import time
import random
import numpy as np
import open3d as o3d



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


#### 参数设置

In [2]:
# 文件存储地址主目录
base_file_path = '/home/akira/桌面/PointCloudCropDemo'
# ShapeNet 选中的椅子
file_path = '/home/akira/下载/Pointnet2_PyTorch-master/PCT/Point-Transformers-master/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/02691156/1c27d282735f81211063b9885ddcbb1.txt' 

# ModelNet 飞机相关的目录
airplane_file = 'ShapeNet_Chair'
airplane_path = os.path.join(base_file_path, airplane_file)

In [3]:
airplane_path

'/home/akira/桌面/PointCloudCropDemo/ShapeNet_Chair'

#### 采样结果可视化函数

In [4]:
def visualize_samples(d_sample):
    print("Points in downsample set: ", len(d_sample))
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0 ,111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    o3d.visualization.draw_geometries([source])

#### 保存点云txt文件

In [5]:
def save_txt(d_sample, file_name_now):
    # 当前是飞机的
    file_path = airplane_path
    now_txt = file_name_now + '.txt'
    txt_file_name = os.path.join(file_path, now_txt)
    np.savetxt(txt_file_name, d_sample, fmt='%.6e', delimiter=',')
    print("Text Saved!")

#### 保存截图PNG文件

In [6]:
def save_png(d_sample, file_name_now):
    # 调整好角度，按下按键S就会拍照并保存
    # get file name
    file_path = airplane_path
    now_png = file_name_now + '.png'
    png_file_name = os.path.join(file_path, now_png)
    
    source = o3d.geometry.PointCloud()
    source.points = o3d.utility.Vector3dVector(d_sample)
    color = [102.0 / 255.0, 111.0 / 255.0, 142.0 / 255.0]
    source.paint_uniform_color(color)
    
    def save_capture_screen_image(vis):
        vis.capture_screen_image(png_file_name)
        return False
    key_to_callback = {}
    key_to_callback[ord("S")] = save_capture_screen_image
    o3d.visualization.draw_geometries_with_key_callbacks([source], key_to_callback)
    print("PNG Saved!")

In [7]:
def save_txt_and_png(d_sample):
    now = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
    save_png(d_sample, now)
    save_txt(d_sample, now)

#### FPS-计算重心

In [8]:
def fps_center(original, npoints):
    center_xyz = np.sum(original, 0)
    # 得到重心点的坐标
    center_xyz = center_xyz / len(original)
    # 计算出初始的最远点
    dist = np.sum((original - center_xyz) ** 2, 1)
    farthest = np.argmax(dist)
    distance = np.ones(len(original)) * 1e10
    target_index = np.zeros(npoints, dtype=np.int32)

    for i in range(npoints):
        target_index[i] = farthest
        target_point_xyz = original[target_index[i], :]
        
        dist = np.sum((original - target_point_xyz) ** 2, 1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance)
    
    return original[target_index]

#### FPS-不计算重心

In [9]:
def fps_random(original, npoints):
    farthest = np.random.randint(len(original))
    distance = np.ones(len(original)) * 1e10
    target_index = np.zeros(npoints, dtype=np.int32)

    for i in range(npoints):
        target_index[i] = farthest
        target_point_xyz = original[target_index[i], :]
        
        dist = dist = np.sum((original - target_point_xyz) ** 2, 1)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = np.argmax(distance)
    
    return original[target_index]

- 原始点云

In [13]:
# 提取并可视化
point_set = np.loadtxt(file_path, delimiter=' ').astype(np.float32)[:, 0:3]
visualize_samples(point_set)

Points in downsample set:  2705


In [50]:
# 保存点云和截图
save_txt_and_png(point_set)

PNG Saved!
Text Saved!


In [14]:
# 初始点取重心，进行一次down sample
downsampling_1st = fps_center(point_set, 1024)
# 可视化
visualize_samples(downsampling_1st)

Points in downsample set:  1024


In [52]:
# 保存
save_txt_and_png(downsampling_1st)

PNG Saved!
Text Saved!


In [57]:
# 初始点随机，给出5个例子
downsampling_2nd = fps_random(point_set, 1024)
save_txt_and_png(downsampling_2nd)

PNG Saved!
Text Saved!


#### 切大Patch

In [15]:
# 大patch（3 slices， 40% per slice）
def get_index(index_slice, ration_per_slice, overlap_ration, num_all_points):
    # index_slice = 0, 1, 2
    start_index = index_slice * (ration_per_slice - overlap_ration) * num_all_points
    end_index = start_index + ration_per_slice * num_all_points
    return int(start_index), int(end_index)

def get_big_patch(point_set, npoints, xyz_dim, index_slice):
    # xyz_dim: 0, 1, 2 for x, y, z
    start_index, end_index = get_index(index_slice, 0.4, 0.1, len(point_set))
    big_patch_index = np.argsort(point_set, axis=0)[start_index: end_index, xyz_dim]
    big_patch = point_set[big_patch_index]
    random.shuffle(big_patch)
    # 返回fps后的值，fps前的值，xyz，index_slice
    return (fps_center(big_patch, npoints), big_patch, xyz_dim, index_slice)

In [16]:
# 测试
point_set = np.loadtxt(file_path, delimiter=' ').astype(np.float32)[:, 0:3]
before_fps, downsampling_big_patch, xyz_dim, index_slice = get_big_patch(point_set, 1024, 2, 2)
visualize_samples(downsampling_big_patch)

Points in downsample set:  1082


#### 新的保存函数，可以保存fps前后的点云

In [60]:
def save_new(bf, d_sample, xyz, sli):
    # xyz 表示取哪个轴
    # sli 表示第几个slice
    if xyz == 0:
        dim = 'x'
    elif xyz == 1:
        dim = 'y'
    else:
        dim = 'z'
    now = dim + '_' + str(sli)
    save_txt(d_sample, now)
    save_png(d_sample, now)
    now_bf = 'Before_fps_' + now
    save_txt(bf, now_bf)
    save_png(bf, now_bf)

In [69]:
# 分别0，1，2 两个值都是，循环这样做就好了
dbp, bf, xyz, sli = get_big_patch(point_set, 1024, 2, 2)
save_new(bf, dbp, xyz, sli)

Text Saved!
PNG Saved!
Text Saved!
PNG Saved!


In [80]:
bf, dbp, xyz, sli = get_big_patch(point_set, 1024, 2, 2)
save_txt_and_png(dbp)

PNG Saved!
Text Saved!


#### 切小Patch

In [17]:
def get_small_patch(point_set, npoints, xyz_dim, index_slice):
    start_index, end_index = get_index(index_slice, 0.2, 0.1, len(point_set))
    small_patch_index = np.argsort(point_set, axis=0)[start_index: end_index, xyz_dim]
    small_patch = point_set[small_patch_index]
    random.shuffle(small_patch)
    return (fps_center(small_patch, npoints), small_patch, xyz_dim, index_slice)

In [24]:
point_set = np.loadtxt(file_path, delimiter=',').astype(np.float32)[:, 0:3]
downsampling_small_patch = get_small_patch(point_set, 1024, 2, 8)
visualize_samples(downsampling_small_patch)

ValueError: could not convert string to float: '0.103360 -0.097800 -0.098760 -0.119900 -0.375500 0.919000 3.000000'

In [130]:
point_set = np.loadtxt(file_path, delimiter=',').astype(np.float32)[:, 0:3]
# 循环0，1，2 + 0-8
ds, bf, axis, sli = get_small_patch(point_set, 1024, 2, 8)
save_new(bf, ds, axis, sli)

Text Saved!
PNG Saved!
Text Saved!
PNG Saved!


#### cube切割

In [20]:
# length setting
space_len = 1.0 + 1.0
cube_length = 0.5 * space_len
step_size = 0.125 * space_len
cube_per_axis = 5

# [-0.5, -0.25, 0.0, 0.25, 0.5]
def compute_axis_coordinate(num, step_size, cube_length):
    result = []
    initial_center = -1.0 + 0.5 * cube_length
    for i in range(num):
        result.append(initial_center)
        initial_center += step_size
    return result

# [125, 3] coordinates of centers
def compute_center_coordiantes():
    # 为了方便这里直接写死了
    axis_coordinate = [-0.5, -0.25, 0.0, 0.25, 0.5]
    result = np.zeros([125, 3])
    for num in range(125):
        t0 = num / 25
        t0 = int(t0) % 5
        result[num][0] = axis_coordinate[t0]
        # y
        t1 = num / 5
        t1 = int(t1) % 5
        result[num][1] = axis_coordinate[t1]
        # z
        t2 = num % 5
        result[num][2] = axis_coordinate[t2]
    return result

def point_in_cube(point_xyz, center_xyz, cube_length):
    flag = 0
    lwh = 0.5 * cube_length
    for i in range(3):
        low = center_xyz[i] - lwh
        high = center_xyz[i] + lwh
        if low<point_xyz[i] and point_xyz[i]<high:
            flag+=1 
    if flag == 3:
        return True
    else:
        return False

def satisfy_one_cube(point_set, npoints, cube_length):
    # 得到所有的cube的中心点
    center_xyzs = compute_center_coordiantes()
    num_all_satisfied = 0
    
    for i in range(len(center_xyzs)):
        output_samples_index = 0
        for j in range(len(point_set)):
            if point_in_cube(point_set[j], center_xyzs[i], cube_length):
                output_samples_index += 1
        if output_samples_index >= npoints:
            num_all_satisfied += 1
        else:
            print("Not enough points here: ", i, " only ", output_samples_index, " points")
    return num_all_satisfied
                
    
def get_downsamplings_by_cube(point_set, cube_index, cube_length=0.1, npoints=1024):
    # 所有中心坐标的集合
    center_xyzs = compute_center_coordiantes()
    # 选出一个中心坐标出来
    center_xyz = center_xyzs[cube_index]
    # 存储满足条件的点的索引
    output_samples = []

    for i in range(len(point_set)):
        if point_in_cube(point_set[i], center_xyz, cube_length):
            output_samples.append(i)
            
    if  len(output_samples) >= npoints:
        samples = point_set[output_samples]
        downsampling = fps_center(samples, npoints)
        return downsampling
    else:
        print("There are not enought number of points in this cube")
    

In [104]:
num = satisfy_one_cube(point_set, 1024, cube_length)
num

Not enough points here:  0  only  261  points
Not enough points here:  1  only  494  points
Not enough points here:  2  only  494  points
Not enough points here:  3  only  494  points
Not enough points here:  4  only  233  points
Not enough points here:  5  only  681  points
Not enough points here:  9  only  645  points
Not enough points here:  10  only  860  points
Not enough points here:  14  only  825  points
Not enough points here:  15  only  827  points
Not enough points here:  19  only  787  points
Not enough points here:  20  only  599  points
Not enough points here:  24  only  592  points
Not enough points here:  25  only  569  points
Not enough points here:  29  only  523  points
Not enough points here:  45  only  825  points
Not enough points here:  49  only  818  points
Not enough points here:  50  only  569  points
Not enough points here:  54  only  523  points
Not enough points here:  70  only  825  points
Not enough points here:  74  only  818  points
Not enough points he

81

In [21]:
# 显示cube中心坐标：
compute_center_coordiantes()

array([[-0.5 , -0.5 , -0.5 ],
       [-0.5 , -0.5 , -0.25],
       [-0.5 , -0.5 ,  0.  ],
       [-0.5 , -0.5 ,  0.25],
       [-0.5 , -0.5 ,  0.5 ],
       [-0.5 , -0.25, -0.5 ],
       [-0.5 , -0.25, -0.25],
       [-0.5 , -0.25,  0.  ],
       [-0.5 , -0.25,  0.25],
       [-0.5 , -0.25,  0.5 ],
       [-0.5 ,  0.  , -0.5 ],
       [-0.5 ,  0.  , -0.25],
       [-0.5 ,  0.  ,  0.  ],
       [-0.5 ,  0.  ,  0.25],
       [-0.5 ,  0.  ,  0.5 ],
       [-0.5 ,  0.25, -0.5 ],
       [-0.5 ,  0.25, -0.25],
       [-0.5 ,  0.25,  0.  ],
       [-0.5 ,  0.25,  0.25],
       [-0.5 ,  0.25,  0.5 ],
       [-0.5 ,  0.5 , -0.5 ],
       [-0.5 ,  0.5 , -0.25],
       [-0.5 ,  0.5 ,  0.  ],
       [-0.5 ,  0.5 ,  0.25],
       [-0.5 ,  0.5 ,  0.5 ],
       [-0.25, -0.5 , -0.5 ],
       [-0.25, -0.5 , -0.25],
       [-0.25, -0.5 ,  0.  ],
       [-0.25, -0.5 ,  0.25],
       [-0.25, -0.5 ,  0.5 ],
       [-0.25, -0.25, -0.5 ],
       [-0.25, -0.25, -0.25],
       [-0.25, -0.25,  0.  ],
       [-0

大概只有19个是不满足的，所以还是可以选择20个出来的

In [23]:
# 可视化
point_set = np.loadtxt(file_path, delimiter=',').astype(np.float32)[:, 0:3]
downsampling_cube = get_downsamplings_by_cube(point_set, 98, 1, 1024)
visualize_samples(downsampling_cube)

There are not enought number of points in this cube


TypeError: object of type 'NoneType' has no len()

In [37]:
# 上面验证完毕，接下来就是重写这个方法，得到fps前后的，然后保存
def get_cube_sample(point_set, cube_index, cube_length, npoints):
    center_xyzs = compute_center_coordiantes()
    center_xyz = center_xyzs[cube_index]
    output_samples = []

    for i in range(len(point_set)):
        if point_in_cube(point_set[i], center_xyz, cube_length):
            output_samples.append(i)
            
    if  len(output_samples) >= npoints:
        samples = point_set[output_samples]
        downsampling = fps_center(samples, npoints)
        # bf, ds, idx, c_xyz
        return (samples, downsampling, cube_index)
    else:
        print("There are not enought number of points in this cube")

In [9]:
def save_cube(bf, ds, idx):
    now_bf = "Before_fps_" + str(idx)
    save_txt(bf, now_bf)
    save_png(bf, now_bf)
    
    now = str(idx)
    save_txt(ds, now)
    save_png(ds, now)

In [66]:
bf, ds, idx = get_cube_sample(point_set, 61, 1, 1024)
save_cube(bf, ds, idx)

Text Saved!
PNG Saved!
Text Saved!
PNG Saved!


### cube for shapenet

In [25]:
# length setting
space_len = 0.5 + 0.5
cube_length = 0.66 * space_len
step_size = 0.33 * space_len
cube_per_axis = 2

def compute_axis_coordinate_sn(cube_per_axis, step_size, cube_length):
    result = []
    initial_center = -0.5 + 0.5 * cube_length
    for i in range(cube_per_axis):
        result.append(initial_center)
        initial_center += step_size
    return result

# [8, 3] coordinates of centers
def compute_center_coordiantes_sn():
    # 为了方便这里直接写死了
    axis_coordinate = [-0.165, 0.165]
    # axis_coordinate = [-0.125, 0.125]
    result = np.zeros([8, 3])
    for num in range(8):
        t0 = num / 4
        t0 = int(t0) % 2
        result[num][0] = axis_coordinate[t0]
        # y
        t1 = num / 2
        t1 = int(t1) % 2
        result[num][1] = axis_coordinate[t1]
        # z
        t2 = num % 2
        result[num][2] = axis_coordinate[t2]
    return result

def point_in_cube(point_xyz, center_xyz, cube_length):
    flag = 0
    lwh = 0.5 * cube_length
    for i in range(3):
        low = center_xyz[i] - lwh
        high = center_xyz[i] + lwh
        if low<point_xyz[i] and point_xyz[i]<high:
            flag+=1 
    if flag == 3:
        return True
    else:
        return False

def satisfy_one_cube(point_set, npoints, cube_length):
    # 得到所有的cube的中心点
    center_xyzs = compute_center_coordiantes_sn()
    num_all_satisfied = 0
    
    for i in range(len(center_xyzs)):
        output_samples_index = 0
        for j in range(len(point_set)):
            if point_in_cube(point_set[j], center_xyzs[i], cube_length):
                output_samples_index += 1
        if output_samples_index >= npoints:
            num_all_satisfied += 1
        else:
            print("Not enough points here: ", i, " only ", output_samples_index, " points")
    return num_all_satisfied
                
    
def get_downsamplings_by_cube_sn(point_set, cube_index, cube_length, npoints):
    # 所有中心坐标的集合
    center_xyzs = compute_center_coordiantes_sn()
    # 选出一个中心坐标出来
    center_xyz = center_xyzs[cube_index]
    # 存储满足条件的点的索引
    output_samples = []

    for i in range(len(point_set)):
        if point_in_cube(point_set[i], center_xyz, cube_length):
            output_samples.append(i)
            
    if  len(output_samples) >= npoints:
        samples = point_set[output_samples]
        downsampling = fps_center(samples, npoints)
        return downsampling
    else:
        print("There are not enought number of points in this cube")

In [26]:
compute_axis_coordinate_sn(cube_per_axis, step_size, cube_length)

[-0.16999999999999998, 0.16000000000000003]

In [27]:
compute_center_coordiantes_sn()

array([[-0.165, -0.165, -0.165],
       [-0.165, -0.165,  0.165],
       [-0.165,  0.165, -0.165],
       [-0.165,  0.165,  0.165],
       [ 0.165, -0.165, -0.165],
       [ 0.165, -0.165,  0.165],
       [ 0.165,  0.165, -0.165],
       [ 0.165,  0.165,  0.165]])

In [28]:
file_path

'/home/akira/下载/Pointnet2_PyTorch-master/PCT/Point-Transformers-master/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/02691156/1c27d282735f81211063b9885ddcbb1.txt'

In [29]:
# 验证
point_set = np.loadtxt(file_path, delimiter=' ').astype(np.float32)[:, 0:3]
downsampling_cube = get_downsamplings_by_cube_sn(point_set, 4, 0.66, 1024)
visualize_samples(downsampling_cube)

Points in downsample set:  1024


In [113]:
# 上面验证完毕，接下来就是重写这个方法，得到fps前后的，然后保存
def get_cube_sample_sn(point_set, cube_index, cube_length, npoints):
    center_xyzs = compute_center_coordiantes_sn()
    center_xyz = center_xyzs[cube_index]
    output_samples = []

    for i in range(len(point_set)):
        if point_in_cube(point_set[i], center_xyz, cube_length):
            output_samples.append(i)
            
    if  len(output_samples) >= npoints:
        samples = point_set[output_samples]
        downsampling = fps_center(samples, npoints)
        # bf, ds, idx, c_xyz
        return (samples, downsampling, cube_index)
    else:
        print("There are not enought number of points in this cube")

In [121]:
bf, ds, idx = get_cube_sample_sn(point_set, 7, 0.66, 1024)
save_cube(bf, ds, idx)

Text Saved!
PNG Saved!
Text Saved!
PNG Saved!


In [90]:
# 7个球，直径为空间总长度1/2，非中心6个球与原边界相切
# 点的坐标：
import numpy as np
ball_center_xyzs = np.zeros((7, 3))
ball_center_xyzs

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [91]:
for i in range(1, len(ball_center_xyzs)):
    if i%2 != 0:
        j = int((i + 1) / 2 - 1)
        print(i, j)
        ball_center_xyzs[i][j] = 0.165
    else:
        j = int(i / 2 - 1)
        print(i, j)
        ball_center_xyzs[i][j] = -0.165
ball_center_xyzs

1 0
2 0
3 1
4 1
5 2
6 2


array([[ 0.   ,  0.   ,  0.   ],
       [ 0.165,  0.   ,  0.   ],
       [-0.165,  0.   ,  0.   ],
       [ 0.   ,  0.165,  0.   ],
       [ 0.   , -0.165,  0.   ],
       [ 0.   ,  0.   ,  0.165],
       [ 0.   ,  0.   , -0.165]])

In [92]:
ball_center_xyzs

array([[ 0.   ,  0.   ,  0.   ],
       [ 0.165,  0.   ,  0.   ],
       [-0.165,  0.   ,  0.   ],
       [ 0.   ,  0.165,  0.   ],
       [ 0.   , -0.165,  0.   ],
       [ 0.   ,  0.   ,  0.165],
       [ 0.   ,  0.   , -0.165]])

In [93]:
# 判断一个点是否在球内
def point_in_ball(point_xyz, center_xyz, radius):
    # point_xyz和center_xyz 分别是点和球心的坐标
    # radius是球的半径
    flag = False
    dist = 0
    for i in range(3):
        dist += (point_xyz[i] - center_xyz[i]) ** 2
    if dist <= radius ** 2:
        flag = True
    return flag

def get_downsamples_by_ball(point_set, ball_index, radius, npoints):
    center_xyz = ball_center_xyzs[ball_index]
    output_samples = []
    for i in range(0, len(point_set)):
        if point_in_ball(point_set[i], center_xyz, radius):
            output_samples.append(i)
    if len(output_samples) >= npoints:
        samples = point_set[output_samples]
        result = fps_center(samples, npoints)
    return (samples, result, ball_index)

def get_ds_ball(point_set, ball_index, radius, npoints):
    center_xyz = ball_center_xyzs[ball_index]
    output_samples = []
    for i in range(0, len(point_set)):
        if point_in_ball(point_set[i], center_xyz, radius):
            output_samples.append(i)
    if len(output_samples) >= npoints:
        samples = point_set[output_samples]
        result = fps_center(samples, npoints)
    else:
        print("Not enough points in this area: ", ball_index)
    return result

In [94]:
ds_test = get_ds_ball_sn(point_set, 3, 0.33, 1024)

In [95]:
def save_txt_and_png(bf, d_sample, ball_index):
    now_fb = 'Before_fps_' + str(ball_index)
    save_png(bf, now_fb)
    save_txt(bf, now_fb)
    
    now = str(ball_index)
    save_png(d_sample, now)
    save_txt(d_sample, now)

In [102]:
point_set = np.loadtxt(file_path, delimiter=' ').astype(np.float32)[:, 0:3]
bf, ds, bix = get_downsamples_by_ball_sn(point_set, 7, 0.33, 1024)
save_txt_and_png(bf, ds, bix)

IndexError: index 7 is out of bounds for axis 0 with size 7